# #2

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [6]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.sort_values(by='sales_value', ascending=False).head(5)

,item_id,sales_value
56233,6534178,467993.62
56193,6533889,42645.75
29195,1029743,37981.91
56228,6534166,31298.96
35054,1082185,27291.02


In [7]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.sort_values(by='user_id', ascending=False).head(5)

,item_id,user_id
35054,1082185,2039
23846,981760,1584
25333,995242,1421
29195,1029743,1345
8056,840361,1319


In [8]:
popularity = data.groupby('item_id')['basket_id'].nunique().reset_index()
popularity.sort_values(by='basket_id', ascending=False).head(5)

,item_id,basket_id
35054,1082185,27362
56233,6534178,18364
29195,1029743,13455
25333,995242,11397
37719,1106523,9175


In [9]:
# В рекомендательных системах корректно разбивать 
# датасет по времени, а не случайно.
## ниже берутся последние 3 недели в качестве тестовой выборки.
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [10]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [11]:
# датафрейм с покупками юзеров на тестовом датасете 
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [12]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [13]:
# Random Recommendation

def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [14]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['random_recommendation_5k'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5000))

result.head()

CPU times: user 16.7 s, sys: 432 ms, total: 17.1 s
Wall time: 17.5 s


,user_id,actual,random_recommendation,random_recommendation_5k
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5564197, 13417090, 1075514, 6704095, 13876669]","[2638618, 958253, 6022376, 895824, 958121, 101..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12188159, 15596417, 6039659, 9932353, 641991]","[14025676, 861309, 2253639, 1504616, 16769888,..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13671759, 1137847, 329555, 953810, 927806]","[955492, 1209716, 8022241, 831536, 1775844, 55..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[966180, 951650, 1169227, 2068190, 992272]","[5567541, 8181182, 10356542, 1129728, 930603, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[858302, 945360, 904362, 903738, 1071632]","[849325, 1203632, 947635, 1018679, 1094919, 71..."


In [15]:
# TopPop

def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)

    recs = popular.head(n).item_id
    return recs.tolist()

In [16]:
%%time

popular_recs = popularity_recommendation(data_train)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result['popular_recommendation_5k'] = result['user_id'].apply(lambda x: popularity_recommendation(data_train, n=5000))

result.head(3)

CPU times: user 9min 56s, sys: 1min 53s, total: 11min 49s
Wall time: 12min 1s


,user_id,actual,random_recommendation,random_recommendation_5k,popular_recommendation,popular_recommendation_5k
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5564197, 13417090, 1075514, 6704095, 13876669]","[2638618, 958253, 6022376, 895824, 958121, 101...","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185, ..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12188159, 15596417, 6039659, 9932353, 641991]","[14025676, 861309, 2253639, 1504616, 16769888,...","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185, ..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13671759, 1137847, 329555, 953810, 927806]","[955492, 1209716, 8022241, 831536, 1775844, 55...","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185, ..."


In [17]:
# Random Weighted Recommendation

def weighted_random_recommendation(items_weights, n=5):
    """Случайные взвешенные рекоммендации
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights.item_id
    weights = np.abs(items_weights.weight)
    recs = np.random.choice(items, p=weights, size=n, replace=False)
    
    return recs.tolist()

In [18]:
popular = data_train.groupby('item_id')['sales_value'].sum().reset_index()
print(round(popular.loc[popular.sales_value < 1].shape[0] / popular.shape[0] * 100), "% of observations have sales_value less than 1")

4 % of observations have sales_value less than 1


In [19]:
%%time

popular.sort_values('sales_value', ascending=False, inplace=True)
popular = popular.loc[popular['sales_value'] > 1]

p = popular['sales_value'].apply(lambda x: np.log(x) / popular.shape[0])

p /= p.sum()
sum(p)
popular['weight'] = p
items_weights = popular[['item_id', 'weight']]

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result['weighted_random_recommendation_5k'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5000))

CPU times: user 19.1 s, sys: 376 ms, total: 19.4 s
Wall time: 20.1 s


In [20]:
result.head(3)

,user_id,actual,random_recommendation,random_recommendation_5k,popular_recommendation,popular_recommendation_5k,weighted_random_recommendation,weighted_random_recommendation_5k
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5564197, 13417090, 1075514, 6704095, 13876669]","[2638618, 958253, 6022376, 895824, 958121, 101...","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185, ...","[960807, 842707, 1138816, 1006488, 985866]","[879977, 8181189, 906854, 830628, 988421, 9883..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12188159, 15596417, 6039659, 9932353, 641991]","[14025676, 861309, 2253639, 1504616, 16769888,...","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185, ...","[13910350, 1080331, 1097707, 932166, 1792709]","[7168289, 1038462, 1002940, 1125681, 865216, 9..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13671759, 1137847, 329555, 953810, 927806]","[955492, 1209716, 8022241, 831536, 1775844, 55...","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185, ...","[938946, 1025682, 1025468, 7025326, 9526564]","[1072524, 9526228, 1008133, 7151824, 12171620,..."


In [21]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

new_cols = result.columns[2:]
for col in new_cols:
    precision = result.apply(lambda row: precision_at_k(row[col], row['actual']), axis=1).mean()
    print(f'Precision@5 {col}: {round(precision, 4)*100}%')

Precision@5 random_recommendation: 0.05%
Precision@5 random_recommendation_5k: 0.04%
Precision@5 popular_recommendation: 15.52%
Precision@5 popular_recommendation_5k: 15.52%
Precision@5 weighted_random_recommendation: 0.08%
Precision@5 weighted_random_recommendation_5k: 0.06999999999999999%


Вывод: качество предсказаний бейзлайнов не улучшается (и может ухудшиться) с семплированием наблюдений (н-р, топ 5000, как в коде выше). 